## Messari standardised subgraphs

https://github.com/messari/subgraphs

### HoneySwap - https://honeyswap.org/

In [1]:
import pandas as pd
import numpy as np
import time
import requests
import datetime as dt

In [2]:
endpoint = 'https://api.thegraph.com/subgraphs/name/messari/honeyswap-gnosis'

6-Month Average Daily Volume

In [3]:
query = """{
  financialsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    id
    protocol{
      name
    }
    dailyVolumeUSD
    dailyTotalRevenueUSD
    dailySupplySideRevenueUSD
    dailyProtocolSideRevenueUSD
    timestamp
    blockNumber
  }
}
"""

In [4]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [5]:
df = pd.json_normalize(data['data']['financialsDailySnapshots'])

In [6]:
df['dailyVolumeUSD'].astype(float).mean()

233417.41843992114

6-Month Average TVL

In [7]:
#DefiLlama https://defillama.com/protocols/dexes 
lma = pd.read_csv("https://api.llama.fi/dataset/honeyswap.csv")

In [8]:
lma

,Unnamed: 0,Date,Timestamp,Honeyswap,Honeyswap.1,Honeyswap.2,Honeyswap.3,Honeyswap.4,Honeyswap.5,Honeyswap.6,...,Honeyswap.287,Honeyswap.288,Honeyswap.289,Honeyswap.290,Honeyswap.291,Honeyswap.292,Honeyswap.293,Honeyswap.294,Honeyswap.295,Honeyswap.296
0,NaN,NaN,NaN,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,...,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes
1,NaN,NaN,NaN,Total,xdai,polygon,Total,Total,Total,Total,...,polygon,polygon,polygon,polygon,polygon,polygon,polygon,polygon,polygon,polygon
2,NaN,NaN,NaN,TVL,TVL,TVL,Tokens(USD),Tokens(USD),Tokens(USD),Tokens(USD),...,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens
3,NaN,NaN,NaN,NaN,NaN,NaN,USDT,MATIC,ROUTE,KRILL,...,AVAX,JPYC,WMATIC,MAWETH,UST,AMDAI,FISH,TEL,HEX,DEFI5
4,NaN,08/04/2021,1.617840e+09,10131675.731226,10131675.731226,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,NaN,14/07/2022,1.657757e+09,6451246.899633457,6382961.627766488,68285.27186696927,471.091903,21.583619972946664,0.024692994542165832,0.00006986987977924645,...,0.17205952628877233,769.3282855820751,85655.53517858822,4.7827775373e-8,0.03629,0.02062176654768191,0.01967429049248775,15.01,206.76353064,NaN
465,NaN,15/07/2022,1.657843e+09,6798382.578888863,6726600.318503418,71782.26038544558,473.005857,21.951989134677895,0.030340271320867435,0.0000709050998771161,...,0.17205952628877233,765.3657437424769,80861.10232049029,4.7827775373e-8,0.03629,0.02062176654768191,0.01967429049248775,15.01,212.3421177,NaN
466,NaN,16/07/2022,1.657930e+09,6924664.16579955,6852864.303474827,71799.8623247231,475.73933507099997,22.28051630075236,0.029343693065802444,0.0000703184893965651,...,0.17205952628877233,765.3657437424769,81638.53938277741,4.7827775373e-8,0.03629,0.02062176654768191,0.01967429049248775,15.01,214.20762082,NaN
467,NaN,17/07/2022,1.658016e+09,7352951.719482331,7277509.976459728,75441.74302260292,477.20652,23.72893429252217,0.03654120268571625,0.00006964301148705956,...,0.17205952628877233,765.3657437424769,81223.00700506796,4.7827775373e-8,0.03629,0.02062176654768191,0.01967429049248775,15.01,223.88855073,NaN


In [11]:
lma.iloc[:,3]

0                   Dexes
1                   Total
2                     TVL
3                     NaN
4            324457.32223
              ...        
301    28461440.367345914
302      28884345.0476636
303     29449453.66706093
304     30363374.27842397
305      29895708.6068861
Name: Solarbeam, Length: 306, dtype: object

In [12]:
#Just eth currently can take total by changing the header
lma.iloc[-183:, 3].astype(float).mean()

27671389.253872823

Largest Pool Concentration

In [21]:
df = pd.DataFrame()
ts = 0

while True:
    sl = len(df)
    query = """
{
  liquidityPools(first:1000, orderBy: createdTimestamp, orderDirection: asc, where: {createdTimestamp_gt: """+f'"{ts}"'+"""}){
    id
    name
    totalValueLockedUSD
    cumulativeVolumeUSD
    createdTimestamp
    createdBlockNumber
  }
}
"""
    request = requests.post(endpoint, json={'query':query})
    data = request.json()

    pool = pd.json_normalize(data['data']['liquidityPools'])
    df = df.append(pool)
    df.drop_duplicates(inplace=True)
    if len(df) == sl:
      break
    ts = df['createdTimestamp'].max()


/tmp/ipykernel_28149/2291472997.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)
/tmp/ipykernel_28149/2291472997.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)
/tmp/ipykernel_28149/2291472997.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)
/tmp/ipykernel_28149/2291472997.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)


In [23]:
df['totalValueLockedUSD'].astype(float).max()/df['totalValueLockedUSD'].astype(float).sum()

0.2407176531159885

In [24]:
df['totalValueLockedUSD'].astype(float).nlargest(10).sum()/df['totalValueLockedUSD'].astype(float).sum()

0.6621316289621388

Liquidity Pools with > $1mn TVL

In [25]:
len(df.loc[df['totalValueLockedUSD'].astype(float) > 1000000])

1

Monthly Active Users

In [15]:
query = """{
  usageMetricsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    timestamp
    blockNumber
    id
    dailyActiveUsers
    dailySwapCount
    dailyDepositCount
    dailyWithdrawCount 
  }
}
"""

In [16]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [17]:
df = pd.json_normalize(data['data']['usageMetricsDailySnapshots'])

In [18]:
df['date'] = df['timestamp'].astype(int).apply(lambda x: dt.datetime.fromtimestamp(x))

In [19]:
monthactiveusers = []
for i in range(len(df['date'].dt.month.unique())):
    monthactiveusers.append(df['dailyActiveUsers'].loc[df['date'].dt.month == df['date'].dt.month.unique()[i]].astype(float).sum())

In [20]:
np.mean(monthactiveusers) #can chart this easy

11067.0

Implied slippage